In [1]:
import boto3
import os

In [ ]:
access_key_id = "minioadmin"
secret_access_key = "minioadmin"
minio_url = "http://localhost:9000"

minio_client = boto3.client(
    "s3",
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    endpoint_url=minio_url
)

new_bucket = "formatted-zone"
try:
    minio_client.head_bucket(Bucket=new_bucket)
except:
    minio_client.create_bucket(Bucket=new_bucket)
    
    
    
name_bucket = "dump"
try:
    minio_client.head_bucket(Bucket=name_bucket)
    print(f"Bucket '{bucket}' ja existeix")
except:
    minio_client.create_bucket(name_bucket)

In [ ]:
import io
from tqdm import tqdm

bucket_origen = "persistent-landing"
bucket_desti = "formatted-zone"
prefix_origen = "textos/"

# Formatos de texto soportados
formatos_texto = {'.txt', '.doc', '.docx', '.pdf', '.rtf', '.odt', '.md'}

paginator = minio_client.get_paginator("list_objects_v2")

for page in paginator.paginate(Bucket=bucket_origen, Prefix=prefix_origen):
    for obj in tqdm(page.get("Contents", []), desc="Processant fitxers de text"):
        key = obj["Key"]
        filename = key.split("/")[-1]
        
        # Obtener la extensión del archivo
        extension = os.path.splitext(filename)[1].lower()
        
        # Solo procesar archivos de texto
        if extension not in formatos_texto:
            # Guardar archivo no reconocido en dump
            response = minio_client.get_object(Bucket=bucket_origen, Key=key)
            file_data = response["Body"].read()
            
            minio_client.put_object(
                Bucket="dump",
                Key=key,
                Body=file_data
            )
            continue
            
        # Crear el nuevo nombre con extensión .txt
        nombre_sin_extension = os.path.splitext(filename)[0]
        nuevo_filename = f"{nombre_sin_extension}.txt"

        # Llegir el contingut del fitxer original
        response = minio_client.get_object(Bucket=bucket_origen, Key=key)
        text = response["Body"].read().decode("utf-8", errors="ignore")

        # Convertir a bytes i pujar-lo al bucket formatted-zone
        new_key = f"textos/{nuevo_filename}"

        minio_client.put_object(
            Bucket=bucket_desti,
            Key=new_key,
            Body=text.encode("utf-8")
        )

Processant fitxers de text: 100%|██████████| 1/1 [00:00<00:00,  5.95it/s]
